# Training Job in Internet-free Mode

If you want to isolate your training data and training container from the rest of the Internet, then you should create the training job in a private subnet. A private subnet is a subnet in your VPC without a route to Internet Gateway. This means, by default, no inbounding calls to your container from the Internet is possible and your container cannot make outbounding calls to the Internet. If you need the training container to access your S3 resource, you need to **explicitly** add a VPC endpoint and attach it to the route table of your private subnet to allow traffic to your data in S3 bucket. 

In this notebook, we will walk through an example of creating such a training job. We will

- Build a simple training image
- Set up a VPC
- Set up a private subnet in the VPC
- Set up a security group in the VPC
- Create a training job in your private subnet && security group and watch it to fail (because it cannot access your S3 resource)
- Add a VPC endpoint to allow traffic to S3
- Create another training job in your private subnet and watch it to succeeed 

If you are not familiar with VPC security configuration, the following materials can help you
- [Security in Amazon Virtual Private Cloud](https://docs.aws.amazon.com/vpc/latest/userguide/security.html)
- [Training and Inference Containers in Internet-Free Mode](https://docs.aws.amazon.com/sagemaker/latest/dg/mkt-algo-model-internet-free.html)

It's okay if you don't understand everything from the official docs above, the code samples you will see in this notebook will help you grasp those concepts. 

In [1]:
# import libraries
import boto3
import pprint
import datetime
import time

pp = pprint.PrettyPrinter(indent=1)

## Build a training image

We follow the same procedure for building a training image as in [this notebook](https://github.com/hsl89/amazon-sagemaker-examples/blob/sagemaker-fundamentals/sagemaker-fundamentals/create-training-job/create-training-job.ipynb). We will refer to this image as `example-image`. 

In [3]:
# create a repo in your ECR 

ecr = boto3.client('ecr')

try:
    # The repository might already exist
    # in your ECR
    cr_res = ecr.create_repository(
        repositoryName='example-image')
    pp.pprint(cr_res)
except Exception as e:
    print(e)

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'example-image' already exists in the registry with id '688520471316'


In [4]:
%%sh
# build the image
cd container/

# tag it as example-image:latest
docker build -t example-image:latest .
    
# test the container
python local_test/test_container.py

account=$(aws sts get-caller-identity --query Account | sed -e 's/^"//' -e 's/"$//')
region=$(aws configure get region)
ecr_account=${account}.dkr.ecr.${region}.amazonaws.com

# Give docker your ECR login password
aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $ecr_account

# Fullname of the repo
fullname=$ecr_account/example-image:latest

# Tag the image with the fullname
docker tag example-image:latest $fullname

# Push to ECR
docker push $fullname

Sending build context to Docker daemon   16.9kB
Step 1/4 : FROM continuumio/miniconda:latest
 ---> b8ea69b5c41c
Step 2/4 : RUN mkdir -p /opt/ml
 ---> Using cache
 ---> a170cc3fed03
Step 3/4 : COPY train.py /usr/bin/train
 ---> Using cache
 ---> 4e19c1cb2076
Step 4/4 : RUN chmod +x /usr/bin/train
 ---> Using cache
 ---> 38609e5aaa0d
Successfully built 38609e5aaa0d
Successfully tagged example-image:latest
== Files in train channel ==
== Files in the test channel ==
== Saving model checkpoint ==
== training completed ==

Login Succeeded
The push refers to repository [688520471316.dkr.ecr.us-west-2.amazonaws.com/example-image]
967579d07803: Preparing
5c8c2d1bcfe6: Preparing
88674bdc7fd9: Preparing
78db50750faa: Preparing
805309d6b0e2: Preparing
2db44bce66cd: Preparing
2db44bce66cd: Waiting
805309d6b0e2: Layer already exists
967579d07803: Layer already exists
5c8c2d1bcfe6: Layer already exists
88674bdc7fd9: Layer already exists
78db50750faa: Layer already exists
2db44bce66cd: Layer already 

WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

